In [10]:
pip install sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.4 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("final_category.csv")

In [4]:
df2=pd.read_csv("features_expand_updated (1).csv")

In [5]:
df1=pd.read_csv("train_image.csv")

In [6]:
df2.head()

,sample_id,catalog_content,unit,unit_qty,pack_count,total_qty,num_bullet_points,num_product_desc,total_chars_bullet_points,total_chars_product_desc,...,price,brand_name,category,price_per_unit,price_per_pack,total_weight_price_ratio,info_density,text_to_price_ratio,avg_char_per_desc,product_type_dynamic
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",ounce,12.00,6,72.00,0.0,0.0,0.0,0.0,...,4.89,La Victoria Green,Food,0.067917,0.815000,0.067917,0.000000,0.000000,0.0,gluten free
1,198967,"Item Name: Salerno Cookies, The Original Butte...",ounce,8.00,4,32.00,5.0,0.0,325.0,0.0,...,13.12,Salerno Cookies,Food,0.410000,3.280000,0.410000,54.166667,23.016997,0.0,spicy
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",ounce,1.90,6,11.40,5.0,0.0,131.0,0.0,...,1.97,Bear Creek Hearty,Food,0.172807,0.328333,0.172807,21.833333,44.107744,0.0,whole grain
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,ounce,11.25,0,11.25,5.0,1.0,512.0,509.0,...,30.34,Judee,Campbell,2.696889,0.000000,0.000000,85.333333,32.578175,254.5,dairy free
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",count,0.00,0,0.00,1.0,0.0,52.0,0.0,...,66.49,Kedem Sherry Cooking,Food,0.000000,0.000000,0.000000,26.000000,0.770485,0.0,premium


In [7]:
df2=df2[["sample_id","info_density","product_type_dynamic"]]

In [8]:
df1.head()

,sample_id,image_link
0,33127,https://m.media-amazon.com/images/I/51mo8htwTH...
1,198967,https://m.media-amazon.com/images/I/71YtriIHAA...
2,261251,https://m.media-amazon.com/images/I/51+PFEe-w-...
3,55858,https://m.media-amazon.com/images/I/41mu0HAToD...
4,292686,https://m.media-amazon.com/images/I/41sA037+Qv...


In [9]:
df.shape

(75000, 14)

In [10]:
final_df=df.merge(df1,how="inner",on="sample_id")

In [11]:
final_df=final_df.merge(df2,how="inner",on="sample_id")

In [12]:
final_df.shape

(75000, 17)

In [13]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoProcessor, AutoModel
from PIL import Image
from io import BytesIO
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sentence_transformers import SentenceTransformer
import joblib
from tqdm import tqdm
import os

image_feature = 'image_link'  # new
sample_id_col = 'sample_id'
def download_and_save_images(df, image_dir="images"):
    os.makedirs(image_dir, exist_ok=True)
    saved_paths = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Downloading images"):
        img_url = row[image_feature]
        sample_id = str(row[sample_id_col])
        ext = os.path.splitext(img_url)[-1].lower()
        if ext not in [".jpg", ".jpeg", ".png"]:
            ext = ".jpg"
        local_path = os.path.join(image_dir, f"{sample_id}{ext}")

        if not os.path.exists(local_path):
            try:
                response = requests.get(img_url, timeout=10)
                image = Image.open(BytesIO(response.content)).convert("RGB")
                image.save(local_path)
            except Exception as e:
                print(f"Warning: Could not download {img_url}: {e}")
                # create blank placeholder
                image = Image.new("RGB", (224, 224), color=(255, 255, 255))
                image.save(local_path)

        saved_paths.append(local_path)
    return saved_paths


In [14]:
import os
import requests
from PIL import Image
from io import BytesIO
from multiprocessing import Pool
from tqdm import tqdm

# Define columns
image_feature = 'image_link'
sample_id_col = 'sample_id'

def _download_image(args):
    """Helper for multiprocessing: downloads a single image."""
    sample_id, img_url, image_dir = args
    if not isinstance(img_url, str) or not img_url.startswith("http"):
        return None  # skip invalid URLs

    ext = os.path.splitext(img_url)[-1].lower()
    if ext not in [".jpg", ".jpeg", ".png"]:
        ext = ".jpg"

    local_path = os.path.join(image_dir, f"{sample_id}{ext}")
    if os.path.exists(local_path):
        return local_path  # already downloaded

    try:
        response = requests.get(img_url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image.save(local_path)
        return local_path
    except Exception as e:
        # create placeholder if failed
        print(f"Warning: Could not download {img_url}: {e}")
        image = Image.new("RGB", (224, 224), color=(255, 255, 255))
        image.save(local_path)
        return local_path


def download_and_save_images(df, sample_ids, image_dir="images", num_workers=16):
    """
    Downloads images only for given sample_ids in parallel.
    Args:
        df (pd.DataFrame): DataFrame containing sample_id and image_link columns.
        sample_ids (list): List of sample IDs to download.
        image_dir (str): Directory to save images.
        num_workers (int): Number of parallel workers.
    Returns:
        list: Paths to all saved images (in the same order as sample_ids).
    """
    os.makedirs(image_dir, exist_ok=True)

    # Filter dataframe to only those sample_ids
    subset_df = df[df[sample_id_col].isin(sample_ids)]
    args_list = [
        (str(row[sample_id_col]), row[image_feature], image_dir)
        for _, row in subset_df.iterrows()
    ]

    print(f"Starting download for {len(args_list)} images using {num_workers} workers...")

    with Pool(num_workers) as pool:
        results = list(
            tqdm(
                pool.imap_unordered(_download_image, args_list),
                total=len(args_list),
                desc="Downloading images"
            )
        )

    print("Downloading complete!")
    return results


In [14]:
download_and_save_images(final_df)

KeyboardInterrupt: 

In [43]:
import os
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm

image_feature = 'image_link'
sample_id_col = 'sample_id'

def download_and_save_images(df, sample_ids, image_dir="images"):
    """
    Downloads images only for the specified sample_ids (sequentially).
    
    Args:
        df (pd.DataFrame): DataFrame containing 'sample_id' and 'image_link' columns.
        sample_ids (list): List of sample IDs whose images should be downloaded.
        image_dir (str): Directory to save the downloaded images.

    Returns:
        list: Local file paths for downloaded (or existing) images.
    """
    os.makedirs(image_dir, exist_ok=True)
    saved_paths = []

    # Filter only the rows that match given sample_ids
    subset_df = df[df[sample_id_col].isin(sample_ids)]

    for _, row in tqdm(subset_df.iterrows(), total=len(subset_df), desc="Downloading images"):
        img_url = row[image_feature]
        sample_id = str(row[sample_id_col])

        # handle missing or invalid URLs
        if not isinstance(img_url, str) or not img_url.startswith("http"):
            print(f"Warning: Invalid image link for sample_id={sample_id}")
            continue

        # Ensure proper file extension
        ext = os.path.splitext(img_url)[-1].lower()
        if ext not in [".jpg", ".jpeg", ".png"]:
            ext = ".jpg"

        local_path = os.path.join(image_dir, f"{sample_id}{ext}")

        # Skip if already exists
        if not os.path.exists(local_path):
            try:
                response = requests.get(img_url, timeout=10)
                image = Image.open(BytesIO(response.content)).convert("RGB")
                image.save(local_path)
            except Exception as e:
                print(f"Warning: Could not download {img_url} (sample_id={sample_id}): {e}")
                # Create placeholder
                image = Image.new("RGB", (224, 224), color=(255, 255, 255))
                image.save(local_path)

        saved_paths.append(local_path)

    return saved_paths

In [ ]:
download_and_save_images(final_df,list((set(final_df["sample_id"].values)-(set(processed_sample_ids)))))

In [15]:
import os
processed_sample_ids=[]
dir= os.listdir("images")


In [17]:
for j in dir:
    processed_sample_ids.append(int(j.split(".")[0]))

In [24]:
len(processed_sample_ids)-75000

-24162

In [ ]:
(set(final_df["sample_id"].values)-(set(processed_sample_ids)))

24162

In [34]:
list(set(final_df["sample_id"].values)-(set(processed_sample_ids)))

24162

In [15]:
import os
dir= os.listdir("images")


In [16]:
len(dir)

75000

In [38]:
for path in list(set(final_df["sample_id"].values)-(set(processed_sample_ids))):
    print(str(path)+".jpg")

131099.jpg
131107.jpg
262183.jpg
39.jpg
46.jpg
131121.jpg
131125.jpg
54.jpg
262198.jpg
262207.jpg
262211.jpg
131144.jpg
77.jpg
111.jpg
112.jpg
121.jpg
262266.jpg
130.jpg
262276.jpg
262277.jpg
262279.jpg
131214.jpg
262287.jpg
131216.jpg
149.jpg
131223.jpg
262301.jpg
262304.jpg
161.jpg
131233.jpg
262316.jpg
175.jpg
180.jpg
131253.jpg
186.jpg
131264.jpg
196.jpg
262345.jpg
131278.jpg
262352.jpg
262355.jpg
131284.jpg
262361.jpg
219.jpg
131297.jpg
131301.jpg
235.jpg
262382.jpg
131322.jpg
262401.jpg
131331.jpg
264.jpg
265.jpg
131339.jpg
131344.jpg
262417.jpg
274.jpg
131347.jpg
131348.jpg
277.jpg
262421.jpg
262426.jpg
283.jpg
262427.jpg
262429.jpg
131358.jpg
131359.jpg
262448.jpg
131377.jpg
262453.jpg
313.jpg
131387.jpg
315.jpg
317.jpg
131390.jpg
323.jpg
262468.jpg
262472.jpg
331.jpg
337.jpg
131412.jpg
349.jpg
262493.jpg
350.jpg
353.jpg
262498.jpg
356.jpg
367.jpg
378.jpg
380.jpg
262527.jpg
384.jpg
262528.jpg
262532.jpg
131463.jpg
262540.jpg
262543.jpg
403.jpg
262548.jpg
262555.jpg
413.jpg
2625

In [40]:
for path in list(set(final_df["sample_id"].values)-(set(processed_sample_ids))):
    path="images/"+str(path)+".jpg"
    if os.path.exists(path):
        os.remove(path)
        print(f"🗑️ Deleted: {path}")
    else:
        print(f"⚠️ File not found: {path}")

🗑️ Deleted: images/131099.jpg
🗑️ Deleted: images/131107.jpg
🗑️ Deleted: images/262183.jpg
🗑️ Deleted: images/39.jpg
🗑️ Deleted: images/46.jpg
🗑️ Deleted: images/131121.jpg
🗑️ Deleted: images/131125.jpg
🗑️ Deleted: images/54.jpg
🗑️ Deleted: images/262198.jpg
🗑️ Deleted: images/262207.jpg
🗑️ Deleted: images/262211.jpg
🗑️ Deleted: images/131144.jpg
🗑️ Deleted: images/77.jpg
🗑️ Deleted: images/111.jpg
🗑️ Deleted: images/112.jpg
🗑️ Deleted: images/121.jpg
🗑️ Deleted: images/262266.jpg
🗑️ Deleted: images/130.jpg
🗑️ Deleted: images/262276.jpg
🗑️ Deleted: images/262277.jpg
🗑️ Deleted: images/262279.jpg
🗑️ Deleted: images/131214.jpg
🗑️ Deleted: images/262287.jpg
🗑️ Deleted: images/131216.jpg
🗑️ Deleted: images/149.jpg
🗑️ Deleted: images/131223.jpg
🗑️ Deleted: images/262301.jpg
🗑️ Deleted: images/262304.jpg
🗑️ Deleted: images/161.jpg
🗑️ Deleted: images/131233.jpg
🗑️ Deleted: images/262316.jpg
🗑️ Deleted: images/175.jpg
🗑️ Deleted: images/180.jpg
🗑️ Deleted: images/131253.jpg
🗑️ Deleted: images/1

In [17]:
final_df.set_index("sample_id")

,catalog_content,unit,unit_qty,pack_count,total_qty,num_bullet_points,num_product_desc,total_chars_bullet_points,total_chars_product_desc,avg_bullet_point_len,price,brand_name,category,image_link,info_density,product_type_dynamic
sample_id,,,,,,,,,,,,,,,,
33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",ounce,12.00,6,72.00,0.0,0.0,0.0,0.0,0.000000,4.890,La Victoria Green,Food,https://m.media-amazon.com/images/I/51mo8htwTH...,0.000000,gluten free
198967,"Item Name: Salerno Cookies, The Original Butte...",ounce,8.00,4,32.00,5.0,0.0,325.0,0.0,65.000000,13.120,Salerno Cookies,Food,https://m.media-amazon.com/images/I/71YtriIHAA...,54.166667,spicy
261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",ounce,1.90,6,11.40,5.0,0.0,131.0,0.0,26.200000,1.970,Bear Creek Hearty,Food,https://m.media-amazon.com/images/I/51+PFEe-w-...,21.833333,whole grain
55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,ounce,11.25,0,11.25,5.0,1.0,512.0,509.0,102.400000,30.340,Judee,Campbell,https://m.media-amazon.com/images/I/41mu0HAToD...,85.333333,dairy free
292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",count,0.00,0,0.00,1.0,0.0,52.0,0.0,52.000000,66.490,Kedem Sherry Cooking,Food,https://m.media-amazon.com/images/I/41sA037+Qv...,26.000000,premium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41424,Item Name: ICE BREAKERS Spearmint Sugar Free M...,ounce,1.50,8,12.00,5.0,0.0,589.0,0.0,117.800000,10.395,Ice Breakers Spearmint,Candy,https://m.media-amazon.com/images/I/81p9PcPsff...,98.166667,sugar free
35537,"Item Name: Davidson's Organics, Vanilla Essenc...",count,0.00,0,0.00,5.0,0.0,1508.0,0.0,301.600000,35.920,Davidson'S Organics,Organic,https://m.media-amazon.com/images/I/51DDKoa+mb...,251.333333,organic
249971,Item Name: Jolly Rancher Hard Candy - Blue Ras...,ounce,5.00,0,5.00,0.0,1.0,0.0,18.0,0.000000,50.330,Jolly Rancher Hard,Candy,https://m.media-amazon.com/images/I/91R2XCcpUf...,0.000000,gluten free


In [17]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoProcessor, AutoModel
from PIL import Image
from io import BytesIO
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sentence_transformers import SentenceTransformer
import joblib
from tqdm import tqdm
# ---------------------------
# SigLIP Embeddings
# ---------------------------
import os
import torch
from transformers import AutoProcessor, AutoModel
from PIL import Image
import numpy as np
from tqdm import tqdm
# ---------------------------
# Columns
# ---------------------------
numeric_features = [
    'unit_qty', 'pack_count', 'total_qty',
    'num_bullet_points', 'num_product_desc',
    'total_chars_bullet_points', 'total_chars_product_desc', 'avg_bullet_point_len',"info_density"
]
categorical_features = ['unit', 'brand_name', "category","product_type_dynamic"]
text_feature = 'catalog_content'
image_feature = 'image_link'  # new

# ---------------------------
# Preprocessor for non-text
# ---------------------------
def build_preprocessor():
    return ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', TargetEncoder(cols=categorical_features, handle_unknown='impute'), categorical_features)
    ])



def batch_generate_siglip_embeddings_from_dir(
    image_dir,
    df,
    text_column='catalog_content',
    device='cuda',
    model_name="google/siglip-base-patch16-224",
    batch_size=16
):
    """
    Generate SigLIP embeddings from images in a directory and text from df based on sample_id.
    Assumes image files are named as {sample_id}.jpg/.png
    
    """

    processor = AutoProcessor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    # Step 1: list all images in directory
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))]

    all_embeddings = []
    batch_images = []
    batch_texts = []

    for idx, img_file in enumerate(tqdm(image_files, desc="Preparing batches from directory")):
        sample_id = os.path.splitext(img_file)[0]  # remove extension
        text = str(df.loc[int(sample_id), text_column])
        img_path = os.path.join(image_dir, img_file)

        # Load image
        image = Image.open(img_path).convert("RGB")

        batch_images.append(image)
        batch_texts.append(text)

        # Process batch
        if len(batch_images) == batch_size or idx == len(image_files) - 1:
            inputs = processor(
                text=batch_texts,
                images=batch_images,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=64
            ).to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                image_embeds = outputs.image_embeds
                text_embeds = outputs.text_embeds
                batch_embeds = (image_embeds + text_embeds) / 2

            all_embeddings.append(batch_embeds.cpu().numpy())

            # Clear batch
            batch_images = []
            batch_texts = []

    return np.vstack(all_embeddings)


# ---------------------------
# Main processing function
# ---------------------------
def process_catalog_df(
    df,
    target_col='price',
    test=False,
    pipeline_path='process_pipeline_image1.pkl',
    model_name='google/siglip-base-patch16-224',
):
    if not test:
        preprocessor = build_preprocessor()
        X_non_text = df[numeric_features + categorical_features]
        y = df[target_col]
        preprocessor.fit(X_non_text,y)
        joblib.dump(preprocessor, pipeline_path)
        print(f"Non-text pipeline saved to {pipeline_path}")
    else:
        preprocessor = joblib.load(pipeline_path)
        print(f"Non-text pipeline loaded from {pipeline_path}")

    # Transform numeric + categorical
    X_non_text_processed = preprocessor.transform(df[numeric_features + categorical_features])
    image_dir = "images"
    # Generate SigLIP image+text embeddings
    siglip_embeddings =  batch_generate_siglip_embeddings_from_dir(
    image_dir=image_dir,
    df=final_df.set_index('sample_id'),  # sample_id as index for lookup
    batch_size=1024,
    device='cuda'
)
    # Combine everything
    X_processed = np.hstack((X_non_text_processed, siglip_embeddings))
    return X_processed


# ---------------------------
# Example Usage
# ---------------------------
# df = pd.read_csv("catalog_data.csv")
# df must include columns:
# ['unit_qty', 'pack_count', 'total_qty', 'num_bullet_points', 'num_product_desc',
#  'total_chars_bullet_points', 'total_chars_product_desc', 'avg_bullet_point_len',
#  'unit', 'brand_name', 'category', 'catalog_content', 'image_link', 'price']

X_train = process_catalog_df(final_df, target_col='price', test=False)
# X_test = process_catalog_df(test_df, target_col='price', test=True)


Non-text pipeline saved to process_pipeline_image1.pkl


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Preparing batches from directory: 100%|██████████| 75000/75000 [47:25<00:00, 26.36it/s]   


In [18]:
X_train.shape


(75000, 781)

In [20]:
Y_train=final_df["price"]

In [21]:
print("X train shape:", X_train.shape)
print("Y train shape:", Y_train.shape)


X train shape: (75000, 781)
Y train shape: (75000,)


In [22]:
X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
Y_train = np.nan_to_num(Y_train, nan=0.0, posinf=0.0, neginf=0.0)


In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.saving import register_keras_serializable

@tf.keras.saving.register_keras_serializable()
def smape_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    numerator = tf.abs(y_true - y_pred)
    denominator = tf.maximum((tf.abs(y_true) + tf.abs(y_pred)) / 2.0, 1e-3)
    smape = numerator / denominator
    smape = tf.where(tf.math.is_finite(smape), smape, tf.zeros_like(smape))
    return tf.reduce_mean(smape)


def build_optimized_mlp(input_dim):
    model = Sequential([
        Dense(512, input_dim=input_dim),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(256),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.25),

        Dense(128),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.2),

        Dense(64, activation='relu'),
        Dropout(0.15),

        Dense(1, activation='linear')
    ])

    optimizer = Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss=smape_loss, metrics=['mae'])
    return model


callbacks = [
    ModelCheckpoint('best_mlp_smape_model_image.h5', monitor='val_loss', save_best_only=True, verbose=1),

]

# ✅ Build and train
input_dim = X_train.shape[1]
model = build_optimized_mlp(input_dim)

history = model.fit(
    X_train, Y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=1
)

# ✅ Evaluate
best_model = tf.keras.models.load_model('best_mlp_smape_model_image.h5', custom_objects={'smape_loss': smape_loss})
# loss, mae = best_model.evaluate(X_test, Y_test, verbose=0)
# print(f"✅ Best Model SMAPE: {loss:.4f}, MAE: {mae:.4f}")


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100


2025-10-13 17:34:19.003691: I external/local_xla/xla/service/service.cc:163] XLA service 0x7b17c4004520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-13 17:34:19.003716: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2025-10-13 17:34:19.100884: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-13 17:34:19.472688: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-10-13 17:34:19.769768: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-13 17:34:19.769804: I external/loca

 87/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0859 - mae: 19.2769 

I0000 00:00:1760376866.704466  423282 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4238 - mae: 9.7438 - val_loss: 0.3875 - val_mae: 8.8037
Epoch 10/100
448/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4224 - mae: 9.7547
Epoch 10: val_loss did not improve from 0.38754
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4223 - mae: 9.7309 - val_loss: 0.3887 - val_mae: 8.9595
Epoch 11/100
454/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4221 - mae: 9.8103
Epoch 11: val_loss did not improve from 0.38754
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4219 - mae: 9.6948 - val_loss: 0.3957 - val_mae: 9.0394
Epoch 12/100
449/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4166 - mae: 9.6026
Epoch 12: val_loss did not improve from 0.38754
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4181 - mae: 9.6191 - val_loss: 0.4103 - val_mae: 9.1178
Epoch 13/100
464/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4177 - mae: 9.6078
Epoch 13: val_loss did not improve from 0.38754
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss:

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4114 - mae: 9.4949 - val_loss: 0.3872 - val_mae: 9.1043
Epoch 18/100
450/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4127 - mae: 9.5420
Epoch 18: val_loss improved from 0.38723 to 0.38508, saving model to best_mlp_smape_model_image.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4101 - mae: 9.4598 - val_loss: 0.3851 - val_mae: 8.9660
Epoch 19/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4061 - mae: 9.3668
Epoch 19: val_loss did not improve from 0.38508
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4088 - mae: 9.4606 - val_loss: 0.4346 - val_mae: 9.2524
Epoch 20/100
453/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4104 - mae: 9.5224
Epoch 20: val_loss improved from 0.38508 to 0.37948, saving model to best_mlp_smape_model_image.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4084 - mae: 9.4258 - val_loss: 0.3795 - val_mae: 8.8079
Epoch 21/100
455/469 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4052 - mae: 9.3150
Epoch 21: val_loss did not improve from 0.37948
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4051 - mae: 9.3872 - val_loss: 0.4388 - val_mae: 9.5209
Epoch 22/100
453/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4044 - mae: 9.4472
Epoch 22: val_loss did not improve from 0.37948
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4050 - mae: 9.3871 - val_loss: 0.4257 - val_mae: 9.2179
Epoch 23/100
447/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4041 - mae: 9.4318
Epoch 23: val_loss improved from 0.37948 to 0.37556, saving model to best_mlp_smape_model_image.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4057 - mae: 9.3867 - val_loss: 0.3756 - val_mae: 8.7527
Epoch 24/100
444/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4001 - mae: 9.1706
Epoch 24: val_loss did not improve from 0.37556
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4024 - mae: 9.3257 - val_loss: 0.4444 - val_mae: 9.6570
Epoch 25/100
459/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4031 - mae: 9.3804
Epoch 25: val_loss did not improve from 0.37556
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4024 - mae: 9.3079 - val_loss: 0.4254 - val_mae: 9.1906
Epoch 26/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3990 - mae: 9.2564
Epoch 26: val_loss did not improve from 0.37556
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4014 - mae: 9.2919 - val_loss: 0.4480 - val_mae: 9.4674
Epoch 27/100
450/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3991 - mae: 9.2271
Epoch 27: val_loss did not improve from 0.37556
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss:

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

# --- Custom SMAPE Loss ---
@tf.keras.saving.register_keras_serializable()
def smape_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    numerator = tf.abs(y_true - y_pred)
    denominator = tf.maximum((tf.abs(y_true) + tf.abs(y_pred)) / 2.0, 1e-3)
    smape = numerator / denominator
    smape = tf.where(tf.math.is_finite(smape), smape, tf.zeros_like(smape))
    return tf.reduce_mean(smape)


# --- Transformer Encoder Block ---
def transformer_encoder(inputs, num_heads=8, ff_dim=512, dropout=0.3):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(x, x)
    attn_output = layers.Dropout(dropout)(attn_output)
    x = layers.Add()([x, attn_output])  # Residual 1

    # Feed-forward block
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(inputs.shape[-1])(ff)
    ff = layers.Dropout(dropout)(ff)
    x = layers.Add()([x, ff])  # Residual 2

    return x


# --- Build Transformer Regression Model ---
def build_transformer_regressor(input_dim):
    inputs = layers.Input(shape=(input_dim,))
    
    # Project to token embeddings (sequence length = 1)
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Reshape((8, 32))(x)  # Example: 8 tokens × 32 dims → feel free to adjust
    
    # Stack of transformer encoders
    for _ in range(3):
        x = transformer_encoder(x, num_heads=4, ff_dim=128, dropout=0.25)
    
    x = layers.GlobalAveragePooling1D()(x)
    
    # Dense head for regression
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(1, activation='linear')(x)
    
    model = models.Model(inputs, outputs, name="TransformerRegressor")

    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=smape_loss, metrics=['mae'])
    return model


# --- Training ---
input_dim = X_train.shape[1]
transformer_model = build_transformer_regressor(input_dim)

transformer_callbacks = [
    callbacks.ModelCheckpoint('best_transformer_smape_model.h5', monitor='val_loss', save_best_only=True, verbose=1),
   
]

history = transformer_model.fit(
    X_train, Y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=transformer_callbacks,
    verbose=1
)

# --- Evaluation ---
best_transformer_model = tf.keras.models.load_model(
    'best_transformer_smape_model.h5',
    custom_objects={'smape_loss': smape_loss}
)

# loss, mae = best_transformer_model.evaluate(X_test, Y_test, verbose=0)
# print(f"✅ Transformer Model SMAPE: {loss:.4f}, MAE: {mae:.4f}")


Epoch 1/100


2025-10-13 17:41:53.515178: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-13 17:41:53.515233: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-13 17:41:53.515249: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-13 17:41:53.515264: I external/l

452/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0746 - mae: 18.7894

2025-10-13 17:42:10.231647: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-13 17:42:10.231696: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-13 17:42:10.231709: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-13 17:42:10.231721: I external/l

469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.0647 - mae: 18.7124

...

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3935 - mae: 9.7393 - val_loss: 0.3782 - val_mae: 9.2284
Epoch 50/100
455/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3903 - mae: 9.5214
Epoch 50: val_loss did not improve from 0.37823
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3929 - mae: 9.7354 - val_loss: 0.4102 - val_mae: 9.5597
Epoch 51/100
456/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3922 - mae: 9.6850
Epoch 51: val_loss did not improve from 0.37823
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3923 - mae: 9.7005 - val_loss: 0.3823 - val_mae: 9.2521
Epoch 52/100
459/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3895 - mae: 9.8284
Epoch 52: val_loss did not improve from 0.37823
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3910 - mae: 9.6755 - val_loss: 0.3800 - val_mae: 9.1912
Epoch 53/100
462/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3929 - mae: 9.7540
Epoch 53: val_loss did not improve from 0.37823
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss:

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3844 - mae: 9.3488 - val_loss: 0.3764 - val_mae: 8.8103
Epoch 61/100
450/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3801 - mae: 9.1551
Epoch 61: val_loss did not improve from 0.37644
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3842 - mae: 9.3364 - val_loss: 0.3896 - val_mae: 8.9782
Epoch 62/100
449/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3844 - mae: 9.2974
Epoch 62: val_loss improved from 0.37644 to 0.37563, saving model to best_transformer_smape_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3839 - mae: 9.2751 - val_loss: 0.3756 - val_mae: 8.7691
Epoch 63/100
463/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3842 - mae: 9.3207
Epoch 63: val_loss improved from 0.37563 to 0.37476, saving model to best_transformer_smape_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3824 - mae: 9.2755 - val_loss: 0.3748 - val_mae: 8.7241
Epoch 64/100
457/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3815 - mae: 9.1506
Epoch 64: val_loss did not improve from 0.37476
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3809 - mae: 9.2123 - val_loss: 0.3838 - val_mae: 8.9856
Epoch 65/100
465/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3833 - mae: 9.1032
Epoch 65: val_loss did not improve from 0.37476
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3807 - mae: 9.1876 - val_loss: 0.3763 - val_mae: 8.7699
Epoch 66/100
461/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3785 - mae: 9.1628
Epoch 66: val_loss did not improve from 0.37476
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3806 - mae: 9.1815 - val_loss: 0.3800 - val_mae: 8.7696
Epoch 67/100
464/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3815 - mae: 9.1367
Epoch 67: val_loss did not improve from 0.37476
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss:

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3748 - mae: 9.0418 - val_loss: 0.3739 - val_mae: 8.6058
Epoch 76/100
461/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3781 - mae: 9.2065
Epoch 76: val_loss did not improve from 0.37387
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3754 - mae: 9.0420 - val_loss: 0.3786 - val_mae: 8.6616
Epoch 77/100
461/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3709 - mae: 9.0381
Epoch 77: val_loss did not improve from 0.37387
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3743 - mae: 9.0125 - val_loss: 0.3783 - val_mae: 8.6753
Epoch 78/100
452/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3712 - mae: 8.8891
Epoch 78: val_loss did not improve from 0.37387
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3723 - mae: 8.9943 - val_loss: 0.3802 - val_mae: 8.6803
Epoch 79/100
455/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3721 - mae: 9.0700
Epoch 79: val_loss did not improve from 0.37387
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: